In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import json
import chess
import math

from agent.mcts import ChessMonteCarloTreeSearch
from agent.predict import MovePredictor
from chess_environment.position_parser import ChessPositionParser

import config

np.random.seed(123)

cfg = config.Config()

Using Theano backend.


In [2]:
# Load ECO file of openings

with open(cfg.resources.eco_file) as f:
    ecos = json.load(f)


In [ ]:
# Load up the trained model
move_predictor = MovePredictor(cfg)
position_parser = ChessPositionParser(cfg, [], [])

elos = [1500]
time_control = '3600+30'
num_simulations = 25

results = []

mcts = ChessMonteCarloTreeSearch(cfg, move_predictor, position_parser, num_simulations=num_simulations)

def save_results(results):
    j = json.dumps(results, indent=2)

    with open('/Users/tom/Projects/Portfolio/web-static/public/chess/openings-strength.json', 'w') as f:
        print(j, file=f)

count = 0
total = len(ecos) * len(elos)

try:
    for opening in ecos:
        for elo in elos:
            fen = opening['f'] + ' - 0 1'
            ab_eval, mcts_eval = mcts.set_position(fen, player_elo=elo, opponent_elo=elo).run_search().get_evals()
            results.append({
                'eco': opening['c'],
                'fen': opening['f'],
                'moves': opening['m'],
                'name': opening['n'],
                'player_elo': elo,
                'ab_eval': ab_eval,
                'mcts_eval': mcts_eval,
            })

            count += 1
            
except KeyboardInterrupt:
    save_results(results)
    raise KeyboardInterrupt
        
save_results(results)

In [4]:
import pandas as pd

df = pd.DataFrame(results)
df['diff_eval'] = df['ab_eval'] - df['mcts_eval']
df['abs_diff_eval'] = df['diff_eval'].abs()

df = df.sort_values(by='abs_diff_eval', ascending=False)

df.head()

,ab_eval,eco,fen,mcts_eval,moves,name,player_elo,diff_eval,abs_diff_eval
1883,0.969745,C37,rnb2bnr/pppp1k1p/5q2/8/4P3/2N1pQ2/PPP3PP/R4RK1...,-0.284953,e2e4 e7e5 f2f4 e5f4 g1f3 g7g5 f1c4 g5g4 e1g1 g...,"King's Gambit Accepted, Double Muzio Gambit, Y...",1500,1.254698,1.254698
2125,-0.201216,C44,r1bqkb1r/pp1p1Npp/2p1nn2/8/2B1P3/8/PPP2PPP/RNB...,0.830207,e2e4 e7e5 g1f3 b8c6 d2d4 c6d4 f3e5 d4e6 f1c4 c...,"Scotch, Cochrane Variation",1500,-1.031424,1.031424
2388,-0.206966,C60,r1b1kbnr/ppp2ppp/2N5/1B1p2q1/4P3/8/PPPP1PPP/RN...,0.725584,e2e4 e7e5 g1f3 b8c6 f1b5 d7d5 f3e5 d8g5 e5c6,"Ruy Lopez: Spanish Countergambit, Harding Gambit",1500,-0.932550,0.932550
2359,0.212702,C58,r1bqkb1r/p4ppp/5n2/np2p1N1/8/5Q2/PPPP1PPP/RNB1...,-0.631178,e2e4 e7e5 g1f3 b8c6 f1c4 g8f6 f3g5 d7d5 e4d5 c...,"Italian Game: Two Knights Defense, Blackburne ...",1500,0.843881,0.843881
1785,0.543423,C30,rnb1k1nr/pppp1ppp/8/4p3/1bB1Pq2/2N2N2/PPPP2PP/...,-0.272510,e2e4 e7e5 f2f4 d8f6 b1c3 f6f4 g1f3 f8b4 f1c4,"King's Gambit Declined: Norwalde Variation, Bu...",1500,0.815932,0.815932


In [9]:
def map_san(moves):
    board = chess.Board()
    moves_san = ''
    for idx, m in enumerate(moves.split(' ')):
        half_move = idx + 1
        
        move = chess.Move.from_uci(m)
        move_san = board.san(move)
        
        if half_move % 2:
            moves_san += '{}. '.format(math.ceil(half_move / 2))
            
        moves_san += '{} '.format(move_san)
        
        board.push(move)
        
    return moves_san

df['moves_san'] = df['moves'].map(map_san)

df.to_pickle('/Users/tom/Projects/Portfolio/data/chess-deep-learning/openings.p')